<a href="https://colab.research.google.com/github/NataKrj/AI-project-2024/blob/main/CompanyAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install google-search-results
!pip install requests beautifulsoup4
!pip install pandas
!pip install fuzzywuzzy
!pip install selenium
!pip install spacy nltk
!python -m spacy download en_core_web_sm
!pip install rapidfuzz
!pip install swifter

  Preparing metadata (setup.py) ... done
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32009 sha256=f2fb12d7fd15cefe77e8eff357e55a97395d1c113bd0732c2b8b1535c6ecc731
  Stored in directory: /root/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 37.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.3/486.3 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 77.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 4

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup, Comment
from concurrent.futures import ThreadPoolExecutor, as_completed
import csv
import re
from requests.exceptions import RequestException, SSLError
import time
from urllib.parse import quote
import random
import spacy
from nltk.sentiment import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

# Load the NLP model
nlp = spacy.load('en_core_web_sm')
sia = SentimentIntensityAnalyzer()

# Load the file
file_path = 'BELGIUM_companies.csv'
df = pd.read_csv(file_path, encoding = 'utf-8' , low_memory=False)
company_names = df['OriginalCompanyName'][0:500].tolist()


# Keywords and associated scores
keywords_score_30 = [
    "sanctions", "criminal", "crime", "corruption", "shell company", "offshore",
    "criminal case", "arrested", "fraud", "money laundering",
    "embezzlement", "terrorism financing", "bribery", "tax evasion",
    "illicit funds", "smuggling", "seized assets", "fines",
    "indictment", "prosecuted", "wanted", "scam", "scandal"
]

keywords_score_5 = [
    "court", "accusation", "penalty", "investigation",
    "insolvency", "violation", "debt", "blackmail", "lawsuit",
    "default", "litigation", "settlement", "audit", "suspicious",
    "foreclosure", "dispute", "breach", "illegal transaction",
    "arbitration", "compliance failure", "tax fraud"
]

keywords_score_minus_1 = ["stock"]  # Negative scoring

score_no_words = 0

# User-Agent Rotation
user_agents = [

    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7)",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36",
    "Mozilla/5.0 (iPhone; CPU iPhone OS 14_0 like Mac OS X)",
    "Mozilla/5.0 (iPad; CPU OS 13_2 like Mac OS X)"
]

def random_headers():
    return {
        "User-Agent": random.choice(user_agents)
    }

# Domains to Exclude
exclude_domains = [
    'dictionary.com', 'wiktionary.org', 'merriam-webster.com',
    'facebook.com', 'twitter.com', 'vimeo.com', 'youtube.com',
    'linkedin.com', 'reddit.com', 'quora.com', 'instagram.com',
    'tiktok.com', 'pinterest.com', 'justia.com'
]

# Keywords to detect dictionary-related content
dictionary_keywords = [
    "definition", "meaning", "dictionary", "thesaurus", "pronunciation"
]

def is_valid_url(url):
    return not any(domain in url for domain in exclude_domains)

# Bing Search Scraper
def bing_search_scrape(company, retries=3, backoff_factor=5):
    query = f'"{company}"'
    url = f"https://www.bing.com/search?q={quote(query)}"

    for attempt in range(retries):
        try:
            response = requests.get(url, headers=random_headers(), timeout=10)

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, "html.parser")
                results = []

                for g in soup.find_all('li', class_='b_algo'):
                    link_tag = g.find('a')
                    if not link_tag or 'href' not in link_tag.attrs:
                        continue

                    link = link_tag['href']
                    title = g.find('h2').text if g.find('h2') else ""
                    snippet = g.find('p').text if g.find('p') else ""

                    if not is_valid_url(link):
                        continue

                    if title or snippet:
                        results.append({
                            "title": title,
                            "link": link,
                            "snippet": snippet
                        })

                # Return results if found
                if len(results) > 0:
                    return results
                else:
                    print(f"No results for {company}, retrying...")

            else:
                print(f"Failed to fetch results for {company} (Attempt {attempt+1}): {response.status_code}")

        except (RequestException, SSLError) as e:
            print(f"Error scraping Bing for {company} (Attempt {attempt+1}): {e}")

        # Exponential backoff before next retry
        time.sleep((attempt + 1) * backoff_factor + random.uniform(2, 5))

    # Return empty if all retries fail
    return []

# Function to check if the content is likely from a dictionary
def is_dictionary_page(soup):
    # Check title
    if soup.title and any(word in soup.title.text.lower() for word in dictionary_keywords):
        return True

    # Check meta description
    meta_description = soup.find("meta", {"name": "description"})
    if meta_description and any(word in meta_description.get("content", "").lower() for word in dictionary_keywords):
        return True

    # Check h1 or h2 headings
    for tag in soup.find_all(['h1', 'h2']):
        if any(word in tag.text.lower() for word in dictionary_keywords):
            return True

    return False


# Extract Relevant Text from URL (Updated)
def extract_text_from_url(url, company_name):
    try:
        response = requests.get(url, headers=random_headers(), timeout=10)
        if response.status_code == 200 and 'text/html' in response.headers.get('Content-Type', ''):
            soup = BeautifulSoup(response.text, 'html.parser')

            # Filter out dictionary pages
            if is_dictionary_page(soup):
                return "Dictionary content skipped"

            # Clean unwanted sections
            for script in soup(["script", "style", "header", "footer", "form", "nav"]):
                script.extract()
            for comment in soup.find_all(string=lambda text: isinstance(text, Comment)):
                comment.extract()

            # Extract relevant text
            relevant_text = ""
            for tag in soup.find_all(['h1', 'h2', 'h3', 'p']):
                if company_name.lower() in tag.text.lower():
                    relevant_text += tag.text + " "

            if not relevant_text:
                relevant_text = ' '.join(soup.stripped_strings)

            return re.sub(r'\s+', ' ', relevant_text[:2000])
        else:
            return "Non-text content skipped"
    except (RequestException, SSLError) as e:
        return f"Request failed for {url}: {e}"

def analyze_text_with_nlp(text):
    doc = nlp(text)

    # Named Entity Recognition (NER)
    entities = [ent.text for ent in doc.ents if ent.label_ in ['ORG', 'PERSON', 'GPE', 'LAW']]

    # Sentiment Analysis
    sentiment_score = sia.polarity_scores(text)['compound']
    sentiment = 'Positive' if sentiment_score > 0.05 else 'Negative' if sentiment_score < -0.05 else 'Neutral'

    return entities, sentiment, sentiment_score

def calculate_score_with_reason(text, snippet, company_name):
    text_lower = text.lower()
    snippet_lower = snippet.lower()
    matching_keywords = []
    score = score_no_words

    # Perform NLP Analysis
    entities, sentiment, sentiment_score = analyze_text_with_nlp(text)

    # Direct sentence match for precision
    def keyword_in_same_sentence(keyword):
        sentences = re.split(r'[.!?]', text_lower)
        for sentence in sentences:
            if company_name.lower() in sentence and keyword in sentence:
                return True
        return False

    # Proximity match for broader detection
    def keyword_near_company(keyword):
        match = re.search(rf"\b{keyword}\b", text_lower)
        if match:
            window = text_lower[max(0, match.start() - 500):match.end() + 500]
            if company_name.lower() in window:
                return True
        return False

    # High-risk keyword scoring
    for keyword in keywords_score_30:
        if keyword_in_same_sentence(keyword):
            matching_keywords.append(keyword)
            score += 30
        elif keyword_near_company(keyword) or keyword in snippet_lower:
            matching_keywords.append(keyword)
            score += 30

    # Medium-risk keyword scoring
    for keyword in keywords_score_5:
        if keyword_in_same_sentence(keyword):
            matching_keywords.append(keyword)
            score += 5
        elif keyword_near_company(keyword) or keyword in snippet_lower:
            matching_keywords.append(keyword)
            score += 5

    # Negative keywords (reduce score)
    for keyword in keywords_score_minus_1:
        if keyword_in_same_sentence(keyword):
            matching_keywords.append(keyword)
            score -= 1
        elif keyword_near_company(keyword) or keyword in snippet_lower:
            matching_keywords.append(keyword)
            score -= 1

    # Boost for financial distress mentions
    if not matching_keywords:
       for term in ["insolvency", "bankruptcy", "liquidation", "dissolved"]:
        if term in text_lower:
            score += 5
            matching_keywords.append(term)

    # Sentiment-based Adjustment
    if sentiment == 'Negative':
        score += 2  # Increase score more significantly for negative sentiment
    elif sentiment == 'Positive' and score > 5:
        score -= 1  # Reduce the score slightly, but not below zero
    elif sentiment == 'Neutral':
        score += 1  # Slightly increase score for neutral sentiment to reflect uncertainty
    return score, matching_keywords or ["No relevant keywords"], entities, sentiment

def clean_text(text):
    return re.sub(r'\s+', ' ', text).strip()

# Process Each Company
def process_company(company_name):
    try:
        results = bing_search_scrape(company_name)
        company_data = []

        # If no results, append a placeholder entry with a score of 0
        if not results:
            return [{
                'company': company_name,
                'url': "N/A",
                'extracted_text': "No results found",
                'score': 0,
                'matched_keywords': "No relevant keywords",
                'entities': "N/A",
                'sentiment': "N/A"
            }]

        # Process results if found
        for result in results:
            url = result['link']
            snippet = result['snippet']
            extracted_text = extract_text_from_url(url, company_name)

            if extracted_text != "Non-text content skipped":
                extracted_text = clean_text(extracted_text)
                score, reasons, entities, sentiment = calculate_score_with_reason(
                    extracted_text, snippet, company_name
                )
                company_data.append({
                    'company': company_name,
                    'url': url,
                    'extracted_text': extracted_text[:300],
                    'score': score,
                    'matched_keywords': ', '.join(reasons),
                    'entities': ', '.join(entities),
                    'sentiment': sentiment
                })

        # If no valid results (filtered by dictionary check), add a placeholder
        if not company_data:
            company_data.append({
                'company': company_name,
                'url': "N/A",
                'extracted_text': "No valid results",
                'score': 0,
                'matched_keywords': "No relevant keywords",
                'entities': "N/A",
                'sentiment': "N/A"
            })

        return company_data

    except Exception:
        # Return error status with a score of 5
        return [{
            "company": company_name,
            "url": "N/A",
            "extracted_text": "Error occurred",
            "score": 0,
            "matched_keywords": "Error",
            "entities": "N/A",
            "sentiment": "N/A"
        }]

# Multithreading for Efficiency (Batching in groups of 50)
data = []
batch_size = 50
total_companies_processed = 0

for i in range(0, len(company_names), batch_size):
    batch = company_names[i:i + batch_size]
    batch_results = []

    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(process_company, name): name for name in batch}
        for future in as_completed(futures):
            try:
                results = future.result()
                batch_results.extend(results)
            except Exception as e:
                print(f"Error during processing batch: {e}")

    # Extend data with the batch results
    data.extend(batch_results)

# Track unique companies
unique_companies_processed = set()

for i in range(0, len(company_names), batch_size):
    batch = company_names[i:i + batch_size]
    batch_results = []

    with ThreadPoolExecutor(max_workers=5) as executor:
        futures = {executor.submit(process_company, name): name for name in batch}
        for future in as_completed(futures):
            results = future.result()
            batch_results.extend(results)
            for entry in results:
                unique_companies_processed.add(entry['company'])

    # Count processed companies for the current batch
    companies_in_batch = len(batch_results)
    total_companies_processed = len(unique_companies_processed)

    print(f"Batch {i // batch_size + 1} completed. {companies_in_batch} companies processed in this batch.")

    # Delay between batches to prevent blocking
    if i + batch_size < len(company_names):
        wait_time = random.randint(45, 75)
        print(f"Waiting {wait_time} seconds before next batch...")
        time.sleep(wait_time)

# Final count of all processed companies
print(f"\nTotal unique companies processed: {total_companies_processed}")
df_results = pd.DataFrame(data)
df_results.to_csv('Step_3.2_company_analysis_with_scores.csv', index=False, encoding='utf-8', quoting=csv.QUOTE_ALL, escapechar='\\')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


No results for Orangette (L'), retrying...
No results for International Lashing Systems nv [INTERNATIONAL LASHING SYSTEMS - MARITIME CONSOLIDATORS BELGIUM], retrying...
No results for International Lashing Systems nv [INTERNATIONAL LASHING SYSTEMS - MARITIME CONSOLIDATORS BELGIUM], retrying...
No results for International Lashing Systems nv [INTERNATIONAL LASHING SYSTEMS - MARITIME CONSOLIDATORS BELGIUM], retrying...
No results for JGI-Computers, retrying...
No results for Just Communication, retrying...
No results for Just Communication, retrying...
No results for Lorsom/Marc, retrying...
No results for Lorsom/Marc, retrying...
No results for Lorsom/Marc, retrying...
No results for Oh ! La ! La ! Boulangerie [OH] LA] LA] BOULANGERIE SPRL], retrying...
No results for Oh ! La ! La ! Boulangerie [OH] LA] LA] BOULANGERIE SPRL], retrying...
No results for Oh ! La ! La ! Boulangerie [OH] LA] LA] BOULANGERIE SPRL], retrying...
No results for Delvith [DELVITH AG], retrying...
No results for D